# ST446 Distributed Computing for Big Data
## Assignment 2 - PART 1
---

*We highly recommend using GCP, as the data sets used are about 20 GB in total.* Alternatively, you can use your own computer.

## P1: Querying the YAGO semantic knowledge base

YAGO is a semantic knowledge base, derived from Wikipedia, WordNet and GeoNames. YAGO contains knowledge about more than 10 million entities (like persons, organizations and cities) and contains more than 120 million facts about these entities. You may find more about YAGO [here](https://www.mpi-inf.mpg.de/departments/databases-and-information-systems/research/yago-naga/yago/#c10444).

In this assignment, you are asked to use parts of the YAGO dataset to demonstrate your knowledge about Spark graphframes and motif queries. In particular, you are asked to **_use motif queries_** to find out answers to the following queries stated in English:

**A (max points 5)**. _Politicians who are also scientists_ (sorted alphabetically by name of person)

**B (max points 5)**. _Companies whose founders were born in London_ (sorted alphabetically by name of founder)

**C (max points 5)**. _Writers who have won a Nobel Prize (in any discipline)_ (sorted alphabetically by name of person)

**D (max points 5)**. _Nobel prize winners who were born in the same city as their spouses_ (sorted alphabetically by name of person)

**E (max points 5)**. _Politicians that are affiliated with a right-wing party_ (sorted alphabetically by name of person)

Please always show the first 20 entries of the resulting DataFrame and the total count of relevant entries.

#### My cluster config
```shell
gcloud beta dataproc clusters create st446-mycluster 
    --properties="^#^spark:spark.jars.packages=graphframes:graphframes:0.5.0-spark2.1-s_2.11,com.databricks:spark-xml_2.11:0.4.1" 
    --subnet default --enable-component-gateway 
    --region europe-west2 --zone europe-west2-c 
    --master-machine-type n1-standard-4 --master-boot-disk-size 500 
    --num-workers 2 --worker-machine-type n1-standard-4 --worker-boot-disk-size 500 
    --image-version 1.3-debian10 --optional-components ANACONDA,JUPYTER 
    --metadata "PIP_PACKAGES=sklearn nltk pandas graphframes" 
    --project st446-lse-2021 --bucket st446-mybucket
```

## 0.1 Get YAGO data

You will need to download the following datasets that are part of YAGO (see [here](https://www.mpi-inf.mpg.de/departments/databases-and-information-systems/research/yago-naga/yago/downloads/) for more information):

* A set of relationships between instances (for example, specifying that Emomali Rahmon is the leader of the Military of Tajikistan). Link: http://resources.mpi-inf.mpg.de/yago-naga/yago3.1/yagoFacts.tsv.7z

* A set of subclass relationships (for example, specifying that *A1086* is *a road in England*, or that *Salmonella Dub* is *a Reggae music group* and also a *New Zealand dub musical group*). Link: http://resources.mpi-inf.mpg.de/yago-naga/yago3.1/yagoTransitiveType.tsv.7z

Please use `wget` to download the data to your compute engine (the files are big!).

Next, you will need to extract `tsv` files from the `7z` archives that you have downloaded. Use the following commands to install `p7zip` on your compute engine and extract the files.
```
sudo apt-get install p7zip-full
7z x yagoTransitiveType.tsv.7z 
7z x yagoFacts.tsv.7z 
```
Please note that this can take a while, in particular as `yagoTransitiveType.tsv` is **18GB** large.

Put the files (`yagoTransitiveType.tsv` and `yagoFacts.tsv`) into the Hadoop file system. 
Also, have a look at their first few lines to understand what kind of data they contain.

***
**The command I used to download data and put data into hdfs**
```
wget http://resources.mpi-inf.mpg.de/yago-naga/yago3.1/yagoFacts.tsv.7z
wget http://resources.mpi-inf.mpg.de/yago-naga/yago3.1/yagoTransitiveType.tsv.7z
sudo apt-get install p7zip-full
7z x yagoTransitiveType.tsv.7z 
7z x yagoFacts.tsv.7z
hadoop fs -put ./ /yago
```

## 0.2 Read the data into a Spark DataFrame

Please load the data from `yagoFacts.tsv` into a DataFrame called `df` and `yagoTransitiveType.tsv` into a DataFrame called `df_subclasses`.
Have a look at the beginning of the files to understand the schema.
Once imported, both DataFrames should have columns labelled as `id`, `subject`, `predicate`, `object` and `value`.
In the case of `yagoTransitiveType.tsv`, some of the predicates can be understood as *"is a sublcass of"* or *"is member of the class"*, and the objects can be understood as classes.

In [1]:
from pyspark.sql.types import *

# your code
# define a schema for the dataframes
schema_df = StructType([
    StructField("id", StringType()),
    StructField("subject", StringType()),
    StructField("predicate", StringType()),
    StructField("object", StringType()),
    StructField("value", DoubleType())])

schema_df_sub = StructType([
    StructField("id", StringType()),
    StructField("subject", StringType()),
    StructField("predicate", StringType()),
    StructField("object", StringType()),
    StructField("value", DoubleType())])

# create dataframe directly from the tsv file on HDFS
df = spark.read.csv('/yago/yagoFacts.tsv', schema=schema_df, header='false', sep='\t')
df_subclasses = spark.read.csv('/yago/yagoTransitiveType.tsv', schema=schema_df_sub, header='false', sep='\t')

## 0.3 Understand the database schema

Let's look at the schema:

In [2]:
# print the table schema
df.printSchema()
df_subclasses.printSchema()

root
 |-- id: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- predicate: string (nullable = true)
 |-- object: string (nullable = true)
 |-- value: double (nullable = true)

root
 |-- id: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- predicate: string (nullable = true)
 |-- object: string (nullable = true)
 |-- value: double (nullable = true)



The useful information is in columns "subject", "predicate" and "object". "predicate" defines the relation between entities "subject" and "object". For example, for "Albert Einstein was born in Ulm", "Albert Einstein" is the subject, "was born in" is the predicate and "Ulm" is the object.

## 0.4 Simple query example

To get information about where Albert Einstein was born, we load data into Spark using the following query:

In [4]:
born_city_df = df.where("predicate = '<wasBornIn>'")
born_city_df.show(1)

+--------------------+--------------------+-----------+---------------+-----+
|                  id|             subject|  predicate|         object|value|
+--------------------+--------------------+-----------+---------------+-----+
|<id_thPX9b1zg!_7f...|<William_Jones_(W...|<wasBornIn>|<Penrhiwceiber>| null|
+--------------------+--------------------+-----------+---------------+-----+
only showing top 1 row



In [5]:
born_city_df.where("subject = '<Albert_Einstein>'").show()

+--------------------+-----------------+-----------+------+-----+
|                  id|          subject|  predicate|object|value|
+--------------------+-----------------+-----------+------+-----+
|<id_sbCVliqDT2_7f...|<Albert_Einstein>|<wasBornIn>| <Ulm>| null|
+--------------------+-----------------+-----------+------+-----+



You may wonder how one would know whether to use the predicate '&lt;wasBornIn&gt;' or '&lt;was_born_in&gt;' and subject '&lt;Albert_Einstein&gt;' or '&lt;AlbertEinstein&gt;'. For YAGO subjects (and objects), the naming is aligned with Wikipedia. For example, Albert Einstein's wiki is: https://en.wikipedia.org/wiki/Albert_Einstein and you can see it is 'Albert_Einstein'. 

For predicates, you can look at the "property" list from the [Yago Web interface](https://gate.d5.mpi-inf.mpg.de/webyagospotlx/WebInterface?L01=%3Fx&L0R=%3CwasBornIn%3E&L02=%3Fc&L0T=&L03=&L0L=&L04=&L05=&L11=&L1R=&L12=&L1T=&L13=&L1L=&L14=&L15=&L21=&L2R=&L22=&L2T=&L23=&L2L=&L24=&L25=&L31=&L3R=&L32=&L3T=&L33=&L3L=&L34=&L35=&L41=&L4R=&L42=&L4T=&L43=&L4L=&L44=&L45=). 
Try different queries with this web interface query to understand more how to query YAGO.

## 0.5 Simple motif example

In this part of the assignment, you are required to use **motif** to find out answer to the 4 questions. Please complete the following example to find out: "Which city was Albert Einstein born in?" using motif queries instead of  SQL queries on the first dataframe (`df`):

In [3]:
from graphframes import *

def vertices_edges_split(df, condition):
    sub = df.filter(condition).select("subject").withColumnRenamed("subject","id")
    obj = df.filter(condition).select("object").withColumnRenamed("object","id")
    v = sub.union(obj).distinct()
    e = df.filter(condition).select("subject","object","predicate")\
    .withColumnRenamed("subject","src").withColumnRenamed("object","dst")
    return v, e

In [3]:
v, e = vertices_edges_split(df, "subject='<Albert_Einstein>'")
g = GraphFrame(v, e)

In [ ]:
# your code
res = g.find("(a)-[e]->(b)").filter("e.predicate = '<wasBornIn>'")
res.show()

## 0.6 Some useful tips

### Get a subset of YAGO database
YAGO database is large, so we don't try to load the entire database into a dataframe and then query it. If you do this, you will find that you won't even be able to execute `df.take(1)`, as it would take up too much of space (at least on a laptop). Instead, you use Spark SQL commands or `df.where` to get a suitable fraction of the data.

### Try the queries in the YAGO Web interface first
It is sometimes tricky to get the right "subject", "predicate" and "object". It is easier if you start from [Yago Web interface](https://gate.d5.mpi-inf.mpg.de/webyagospotlx/WebInterface?L01=%3Fx&L0R=%3CwasBornIn%3E&L02=%3Fc&L0T=&L03=&L0L=&L04=&L05=&L11=&L1R=&L12=&L1T=&L13=&L1L=&L14=&L15=&L21=&L2R=&L22=&L2T=&L23=&L2L=&L24=&L25=&L31=&L3R=&L32=&L3T=&L33=&L3L=&L34=&L35=&L41=&L4R=&L42=&L4T=&L43=&L4L=&L44=&L45=) rather than directly querying in Pyspark. Once your query works, you can convert your query to Pyspark code. Note that sometimes the web version of object/subject code may be different from what you need to type here. For example, company code is &lt;wordnet_company_108058098&gt; when you do the query here but when you do it via the web interface it is &lt;wordnet company 108058098&gt;. 

### Be patient and don't do this exercise in the last minute
Some trial and error is needed to get the query right and it may take some time get the result for a query. For these reasons, we advise you not to wait to work out this exercise just before the submission deadline. 

### Make sure to get the initialization actions right
For this exercise, you will be using GraphFrames.

## 1. Politicians who are also scientists (Question A)
Find all politicians who are also scientists. Output top 20 of them. How many people are in the dataset who are both scientists and politicians?
Please follow these steps:
* Operate on the subsets of `df_subclasses` where the objects are `'<wordnet_scientist_110560637>` (scientists) and `'<wordnet_politician_110450303>'` (politicians), and where the predicates are `rdf:type`.
* Use graphframes and the right parts of `df_subclasses` to construct a graph whose (directed) edges point from subjects to objects. Hence, its source vertices are subjects and it destination vertices are objects. It may be convenient to use intermediate DataFrames and join all the required dataframes of edges and vertices.
* The subjects will be people and the objects will be classes (e.g., scientists, politicians).
* Use a motif query to find all instances that fulfil the criteria specified in the question.
* It is a good idea to define a function that takes a DataFrame and outputs a set of data frames for vertices and edges.

Please sort the output alphabetically by the person column.

***
#### My solutions
A main challenge (for each part) besides the motiv query is to first find a suitable subset of the Yago data to create a graph, which can then be processed using the motiv queries. This subset should not be too large because that would lead to a very long processing time for the motiv query requires.

For this first question, the subset I use includes:
- all scientists, and
- all politians, 

as well as the information about all the relationships which stored in the edges (who is a scientist and who is a politian).

In [64]:
# define condiions
condition1 = "object='<wordnet_scientist_110560637>'"
condition2 = "object='<wordnet_politician_110450303>'"

# create graph
v1, e1 = vertices_edges_split(df_subclasses, condition1)
v2, e2 = vertices_edges_split(df_subclasses, condition2)

v = v1.union(v2).distinct()
e = e1.union(e2).distinct()
g = GraphFrame(v, e)

# define motiv query
res = g.find("(a)-[e1]->(b); (a)-[e2]->(c)").filter("b.id < c.id")

In [ ]:
%%time
res.orderBy("a").distinct().show(20)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                  e1|                   b|                  e2|                   c|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|      [<A._C._Cuza>]|[<A._C._Cuza>, <w...|[<wordnet_politic...|[<A._C._Cuza>, <w...|[<wordnet_scienti...|
|[<A._P._J._Abdul_...|[<A._P._J._Abdul_...|[<wordnet_politic...|[<A._P._J._Abdul_...|[<wordnet_scienti...|
|       [<Aad_Kosto>]|[<Aad_Kosto>, <wo...|[<wordnet_politic...|[<Aad_Kosto>, <wo...|[<wordnet_scienti...|
|        [<Aad_Nuis>]|[<Aad_Nuis>, <wor...|[<wordnet_politic...|[<Aad_Nuis>, <wor...|[<wordnet_scienti...|
| [<Aaron_Aaronsohn>]|[<Aaron_Aaronsohn...|[<wordnet_politic...|[<Aaron_Aaronsohn...|[<wordnet_scienti...|
|  [<Aaron_Farrugia>]|[<Aaron_Farrugia>...|[<wordnet_politic...|[<Aaron_Farrugia>...|[<wordnet_scienti...|
|        [<Ab_Klink>]|[<Ab_Klink>, <w

In [ ]:
%%time
res.count()

CPU times: user 144 ms, sys: 21.8 ms, total: 166 ms
Wall time: 10min 36s


7182

The total number of politicians that are also scientists is: 7182

## 2. Companies whose founders were born in London (Question B)
For companies, use `'<wordnet_company_108058098>'`. 
For *"being founder"*, use `<created>`.

By now, you will understand which DataFrame to use for what. 

Set up a graph and use a motif query to find companies whose founders were born in London.
Please take some time to figure out how a suitable configuration of nodes and edges should look like.  How many such companies are there in our dataset?

Please sort the output alphabetically by the founder column.

The subset used for this part consists of:
- all companies,
- all founders and their companies, and
- all people born in London,

as well as, again, all relationships that are present in the subset of the data.

In [11]:
# define condiions
condition1 = "object='<wordnet_company_108058098>'"
condition2 = "predicate='<created>'"
condition3 = "predicate='<wasBornIn>' AND object='<London>'"

# create graph
v1, e1 = vertices_edges_split(df_subclasses, condition1)
v2, e2 = vertices_edges_split(df, condition2)
v3, e3 = vertices_edges_split(df, condition3)

v = v1.union(v2).union(v3).distinct()
e = e1.union(e2).union(e3).distinct()
g = GraphFrame(v, e)

# define motiv query
res = g.find("(company)-[e1]->(wordnet);(founder)-[e2]->(company);(founder)-[e3]->(city)")\
       .filter("e1.dst='<wordnet_company_108058098>'")\
       .filter("e2.predicate='<created>'")\
       .filter("e3.dst='<London>'")

In [ ]:
%%time
res.orderBy("founder").show(20)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|             company|                  e1|             wordnet|             founder|                  e2|                  e3|      city|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|     [<Dare_Comics>]|[<Dare_Comics>, <...|[<wordnet_company...|      [<Adam_Hamdy>]|[<Adam_Hamdy>, <D...|[<Adam_Hamdy>, <L...|[<London>]|
|[<Jawbone_(compan...|[<Jawbone_(compan...|[<wordnet_company...|[<Alexander_Assei...|[<Alexander_Assei...|[<Alexander_Assei...|[<London>]|
|      [<Video_Arts>]|[<Video_Arts>, <w...|[<wordnet_company...|      [<Antony_Jay>]|[<Antony_Jay>, <V...|[<Antony_Jay>, <L...|[<London>]|
|[<SENS_Research_F...|[<SENS_Research_F...|[<wordnet_company...|  [<Aubrey_de_Grey>]|[<Aubrey_de_Grey>...|[<Aubrey_de_Grey>...|[<London>]|
|[<Andreessen_Horo...|[<And

There are only 59 different companies, but 61 entries

In [ ]:
%%time
# show count of distinct companies
res.select("company").distinct().count()

CPU times: user 146 ms, sys: 52.6 ms, total: 199 ms
Wall time: 12min 17s


59

In [ ]:
%%time
res.count()

CPU times: user 124 ms, sys: 45.3 ms, total: 169 ms
Wall time: 10min 32s


61

## 3. Writers who have won a Nobel Prize in any discipline, including economics (Question C)
Tags for nobel prizes look like these: `'<Nobel_Prize_in_Chemistry>`, `<Nobel_Prize_in_Physics>'`, `<Nobel_Prize>` or `<Nobel_Prize>` etc.
We are also counting this one: `'<Nobel_Memorial_Prize_in_Economic_Sciences>'`.

The tag for writers is `'<wordnet_writer_110794014>'`.

You will need to use `'<hasWonPrize>'` as a predicate.

Please sort the output alphabetically by the person column.

Subset includes:
- all writers, and
- all people who have one any of the Nobel Prizes.

In [5]:
# all nobel prices
prizes = """('<Nobel_Prize_in_Chemistry>','<Nobel_Prize_in_Physics>','<Nobel_Prize>',\
            '<Nobel_Memorial_Prize_in_Economic_Sciences>','<Nobel_Peace_Prize>',\
            '<Nobel_Prize_in_Physiology_or_Medicine>','<Nobel_Prize_in_Literature>')"""

# define conditions
condition1 = "object='<wordnet_writer_110794014>'"
condition2 = "predicate='<hasWonPrize>' AND object IN {}".format(prizes)

# create graph
v1, e1 = vertices_edges_split(df_subclasses, condition1)
v2, e2 = vertices_edges_split(df, condition2)

v = v1.union(v2).distinct()
e = e1.union(e2).distinct()
g = GraphFrame(v, e)

# define motiv query
res = g.find("(writer)-[e1]->(price); (writer)-[e2]->(type)")\
       .filter("e1.predicate='<hasWonPrize>' AND e2.dst='<wordnet_writer_110794014>'")

In [ ]:
%%time 
res.orderBy("writer").show(20)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|              writer|                  e1|               price|                  e2|                type|
+--------------------+--------------------+--------------------+--------------------+--------------------+
| [<14th_Dalai_Lama>]|[<14th_Dalai_Lama...|[<Nobel_Peace_Pri...|[<14th_Dalai_Lama...|[<wordnet_writer_...|
|[<Adrienne_Clarks...|[<Adrienne_Clarks...|[<Nobel_Prize_in_...|[<Adrienne_Clarks...|[<wordnet_writer_...|
|         [<Al_Gore>]|[<Al_Gore>, <Nobe...|[<Nobel_Peace_Pri...|[<Al_Gore>, <word...|[<wordnet_writer_...|
|    [<Albert_Camus>]|[<Albert_Camus>, ...|[<Nobel_Prize_in_...|[<Albert_Camus>, ...|[<wordnet_writer_...|
| [<Albert_Einstein>]|[<Albert_Einstein...|[<Nobel_Prize_in_...|[<Albert_Einstein...|[<wordnet_writer_...|
|   [<Albert_Lutuli>]|[<Albert_Lutuli>,...|[<Nobel_Peace_Pri...|[<Albert_Lutuli>,...|[<wordnet_writer_...|
|[<Albert_Schweitz...|[<Albert_Schwei

In [ ]:
%%time
res.count()

CPU times: user 68.5 ms, sys: 38.7 ms, total: 107 ms
Wall time: 7min 47s


260

## 4. Nobel prize winners who were born in the same city as their spouses (Question D)
You may find the predicate `'<isMarriedTo>'` useful to create a Dataframe of all mariages.
Please also show the cities in which the Nobel laureates and their spouses were born.

Please sort the output alphabetically by the person (prize winner) column.

In short, the yago subset includes:
- all Nobel Price winners (any price),
- all marriages, and
- all birth information.

In [7]:
# define condistions
condition1 = "predicate='<hasWonPrize>' AND object in {}".format(prizes)
condition2 = "predicate='<isMarriedTo>'"
condition3 = "predicate='<wasBornIn>'"

# create graph
v1, e1 = vertices_edges_split(df, condition1)
v2, e2 = vertices_edges_split(df, condition2)
v3, e3 = vertices_edges_split(df, condition3)

v = v1.union(v2).union(v3).distinct()
e = e1.union(e2).union(e3).distinct()

g = GraphFrame(v, e)

res = g.find("(winner)-[e1]->(price); (winner)-[e2]->(spouse); (winner)-[e3]->(city); (spouse)-[e4]->(city)")\
       .filter("""e1.predicate='<hasWonPrize>' AND 
               e2.predicate='<isMarriedTo>' AND 
               e3.predicate='<wasBornIn>' AND
               e4.predicate='<wasBornIn>'""")

In [ ]:
%%time
res.orderBy("winner").show(20)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              winner|                  e1|               price|                  e2|              spouse|                  e3|                city|                  e4|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|         [<Al_Gore>]|[<Al_Gore>, <Nobe...|[<Nobel_Peace_Pri...|[<Al_Gore>, <Tipp...|     [<Tipper_Gore>]|[<Al_Gore>, <Wash...|[<Washington,_D.C.>]|[<Tipper_Gore>, <...|
|[<Carl_Ferdinand_...|[<Carl_Ferdinand_...|[<Nobel_Prize_in_...|[<Carl_Ferdinand_...|      [<Gerty_Cori>]|[<Carl_Ferdinand_...|          [<Prague>]|[<Gerty_Cori>, <P...|
| [<Fridtjof_Nansen>]|[<Fridtjof_Nansen...|[<Nobel_Peace_Pri...|[<Fridtjof_Nansen...|      [<Eva_Nansen>]|[<Fridtjof_Nansen...|            [<Oslo>]|[<

In [8]:
%%time
res.count()

CPU times: user 26.8 ms, sys: 7.78 ms, total: 34.6 ms
Wall time: 2min 3s


8

## 5. Politicians that are affiliated with a right-wing party (Question E)

We are looking for all connections of the form `polician -> party`, where party is a right-wing party and politicians are defined above. If one politician is associated with several right wing parties, you may count them several times.

Use `'<isAffiliatedTo>'` to find membership in organisations and `'<wikicat_Right-wing_parties>'` for right-wing parties organisations.

There are multiple ways to do this.

Please sort the output alphabetically by the person (politician) column.

The subset includes:
- all politians,
- all affiliations, and
- all right-wing parties.

In [9]:
# define conditions
condition1 = "object='<wordnet_politician_110450303>'"
condition2 = "predicate='<isAffiliatedTo>'"
condition3 = "predicate='rdf:type' AND object='<wikicat_Right-wing_parties>'"

# create graph
v1, e1 = vertices_edges_split(df_subclasses, condition1)
v2, e2 = vertices_edges_split(df, condition2)
v3, e3 = vertices_edges_split(df_subclasses, condition3)

v = v1.union(v2).union(v3).distinct()
e = e1.union(e2).union(e3).distinct()
g = GraphFrame(v, e)

# define motiv query
res = g.find("(a)-[e1]->(b); (a)-[e2]->(c); (c)-[e3]->(e)")\
       .filter("e1.predicate='rdf:type' AND e1.dst='<wordnet_politician_110450303>'")\
       .filter("e2.predicate='<isAffiliatedTo>'")\
       .filter("e3.dst='<wikicat_Right-wing_parties>'")

In [ ]:
%%time
res.orderBy("a").show(20)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                  e1|                   b|                  e2|                   c|                  e3|                   e|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[<A.N.M._Ehsanul_...|[<A.N.M._Ehsanul_...|[<wordnet_politic...|[<A.N.M._Ehsanul_...|[<Bangladesh_Nati...|[<Bangladesh_Nati...|[<wikicat_Right-w...|
|[<A._A._Wijethunga>]|[<A._A._Wijethung...|[<wordnet_politic...|[<A._A._Wijethung...|[<United_National...|[<United_National...|[<wikicat_Right-w...|
|    [<A._B._Colton>]|[<A._B._Colton>, ...|[<wordnet_politic...|[<A._B._Colton>, ...|[<Republican_Part...|[<Republican_Part...|[<wikicat_Right-w...|
|   [<A._C._Clemons>]|[<A._C._Clemons>,...|[<wordnet_politic...|[<A._C._Clemons>,...|[<Republican_Part...|

In [ ]:
%%time
res.count()

CPU times: user 220 ms, sys: 72.5 ms, total: 293 ms
Wall time: 18min 38s


32243